In [ ]:
!pip install -q diffusers==0.3.0
!pip install -q transformers scipy ftfy
!pip install -q "ipywidgets>=7,<8"

In [1]:
!pip install -q git+https://github.com/keras-team/keras-cv
!pip install -q --upgrade tensorflow==2.9.1

In [2]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [3]:
!nvidia-smi

Mon Sep 26 01:36:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import keras_cv
import time
from tensorflow.keras import mixed_precision
import tensorflow as tf
mixed_precision.set_global_policy("mixed_float16")
supermodel = keras_cv.models.StableDiffusion(jit_compile=True)

In [6]:
start = time.time()
prompt = "a photograph of an astronaut riding a horse"
images = supermodel.text_to_image(prompt, batch_size=3, num_steps=50)
end = time.time()
print('Coldstart time', end - start)

50/50 [==============================] - 64s 558ms/step
Coldstart time 83.47688627243042


In [7]:
start = time.time()
prompt = "a photograph of an astronaut riding a horse"
images = supermodel.text_to_image(prompt, batch_size=3, num_steps=50)
end = time.time()
print('Warmstart time', end - start)

50/50 [==============================] - 28s 566ms/step
Warmstart time 28.9741427898407


In [ ]:
del supermodel
tf.keras.backend.clear_session()

# HuggingFace Diffusers

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import time
from torch import autocast

# make sure you're logged in with `huggingface-cli login`
pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True,
)

pipe = pipe.to("cuda")

{'trained_betas'} was not found in config. Values will be initialized to default values.


In [ ]:
start = time.time()
prompt = ["a photograph of an astronaut riding a horse"] * 3
with autocast("cuda"):
    images = pipe(prompt, height=512, width=512).images
    print(images)
end = time.time()
print('Coldstart time', end - start)

  0%|          | 0/51 [00:00<?, ?it/s]

[<PIL.Image.Image image mode=RGB size=512x512 at 0x7FC06ED7BA90>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7FC06ED7B790>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7FC06ED7BC10>]
Coldstart time 46.27326512336731


In [ ]:
start = time.time()
prompt = ["a photograph of an astronaut riding a horse"] * 3
with autocast("cuda"):
    images = pipe(prompt, height=512, width=512).images
    print(images)
end = time.time()
print('Warmstart time', end - start)

  0%|          | 0/51 [00:00<?, ?it/s]

[<PIL.Image.Image image mode=RGB size=512x512 at 0x7FC0666C4290>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7FC0666C4790>, <PIL.Image.Image image mode=RGB size=512x512 at 0x7FC0666C48D0>]
Warmstart time 41.33208084106445
